<a href="https://colab.research.google.com/github/maggiemcc02/HessianMonitorWork/blob/main/ExternalOperator_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this notebook I explore some simple examples where I make use of ExternalOperator in my weak forms

In particular, I explore which numpy tools work with ExternalOperator without me having to convert between numpy and firedrake objects. I also explore the tools that require me to convert objects between librairies.


Overall, I think these examples can tell me alot about how I will need to use the ExternalOperator to compute my Hessian based monitor function!

## Imports

In [56]:
# install firedrake

# hide output
%%capture

try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [57]:
# Code in this cell makes plots appear an appropriate size and resolution in the browser window

%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (11, 6)

In [58]:
# import firedrake tools
from firedrake import *
import numpy as np
import matplotlib.pyplot as plt # firedrake makes use of matplotlib tools
from firedrake.pyplot import tripcolor, tricontour, triplot #firedrake plotting
from IPython.display import display
from mpl_toolkits import mplot3d
import scipy as sci


In [59]:
import mpi4py
import mpi4py.rc
#mpi4py.rc.threaded = False

# Scalar Functions

## Firedrake Example

In [60]:
# external operator check

class TranslationOperator(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""
      print('Being Used')
      print()
      u, f = self.ufl_operands
      N = assemble(u - f)
      return N

  @assemble_method((1, 0), (0, 1))
  def assemble_Jacobian(self, *args, **kwargs):
      """Evaluate the Jacobian of N"""
      dNdu = Function(self.function_space()).assign(1)

      print("Jacobian being used")
      print()

      # Construct the Jacobian matrix
      integral_types = set(['cell'])
      assembly_opts = kwargs.get('assembly_opts')
      J = self._matrix_builder((), assembly_opts, integral_types)
      print(type(J))
      print()
      with dNdu.dat.vec as vec:
          J.petscmat.setDiagonal(vec)
      print(type(J))
      print()
      return J

In [61]:
mesh = UnitSquareMesh(8, 8)
V = FunctionSpace(mesh, "CG", 1)

u = Function(V)
v = TestFunction(V)
f = Function(V).interpolate(1)
bcs = DirichletBC(V, 0, "on_boundary")

# This Fails - WHY?

# NJ = TranslationOperator(u, f, function_space=V).assemble_Jacobian()
# print(type(NJ))
# print()


N = TranslationOperator(u, f, function_space=V)
F = (inner(grad(u), grad(v)) + inner(N, v)) * dx
solve(F == 0, u, bcs=bcs)

Being Used

Jacobian being used

<class 'firedrake.matrix.Matrix'>

<class 'firedrake.matrix.Matrix'>

Being Used



## Use Numpy Absolute Value in the Weak Form

In [62]:
class AbsVal(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""
      f = self.ufl_operands[0]
      N = np.abs(f)
      return N


In [63]:
mesh = UnitSquareMesh(24, 24)
xi, eta = SpatialCoordinate(mesh)
V = FunctionSpace(mesh, "CG", 1)

f = Function(V).interpolate(xi**2 - 1)

# print(f.dat.data)
# print()

N = AbsVal(f, function_space =  V).assemble_N()
print(N)
print("N is of type", type(N), "without any conversions")

|w₄₃₈|
N is of type <class 'ufl.algebra.Abs'> without any conversions


In [64]:
u = Function(V)
v = TestFunction(V)

bcs = DirichletBC(V, 0, "on_boundary")

F = (inner(grad(u), grad(v)) - inner(N, v)) * dx
solve(F == 0, u, bcs=bcs)

## Use Numpy Multiplication in the Weak Form

In [65]:
# multiply

class MyMult(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""
      a, b = self.ufl_operands
      N = np.multiply(a, b)
      return N

In [66]:
# now lets solve a problem

mesh = UnitSquareMesh(24, 24)
xi, eta = SpatialCoordinate(mesh)
V = FunctionSpace(mesh, "CG", 1)
u = Function(V)
v = TestFunction(V)
f = Function(V).interpolate(xi**2)
g = Function(V).interpolate(2)

bcs = DirichletBC(V, 0, "on_boundary")

N = MyMult(g, f, function_space=V).assemble_N()

print("N is of type", type(N), "without any conversions")


F = (inner(grad(u), grad(v)) - inner(N, v)) * dx
solve(F == 0, u, bcs=bcs)

N is of type <class 'ufl.algebra.Product'> without any conversions


## Use Numpy sin(x) in the Weak Form (Conversions Required!)

In [67]:
class MySin(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""
      my_x = self.ufl_operands[0]


      # N = np.sin(my_x)
      # above fails because of a
      # TypeError: loop of ufunc does not support argument 0 of type Function which has no callable sin method

      N = np.sin(my_x.dat.data) # apparently I need to convert to numpy array !?

      return N

In [68]:
# now lets solve a problem

mesh = UnitSquareMesh(24, 24)
xi, eta = SpatialCoordinate(mesh)
V = FunctionSpace(mesh, "CG", 1)
u = Function(V)
v = TestFunction(V)
my_x = Function(V).interpolate(xi**2)

bcs = DirichletBC(V, 0, "on_boundary")


N = MySin(my_x, function_space=V).assemble_N()

print("N is resturned as a ", type(N), 'so I need to convert it to ufl')


N_new = Function(V)
N_new.dat.data[:] = N[:]


F = (inner(grad(u), grad(v)) - inner(N_new, v)) * dx
solve(F == 0, u, bcs=bcs)

N is resturned as a  <class 'numpy.ndarray'> so I need to convert it to ufl


## Use Numpy exp(x) in the Weak Form (Conversions Required!)

In [69]:
class MyExp(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""
      my_x = self.ufl_operands[0]


      # N = np.exp(my_x)
      # above fails because of a
      # TypeError: loop of ufunc does not support argument 0 of type Function which has no callable exp method

      N = np.exp(my_x.dat.data) # apparently I need to convert to numpy array !?

      return N

In [70]:
# now lets solve a problem

mesh = UnitSquareMesh(24, 24)
xi, eta = SpatialCoordinate(mesh)
V = FunctionSpace(mesh, "CG", 1)
u = Function(V)
v = TestFunction(V)
my_x = Function(V).interpolate(xi**2)

bcs = DirichletBC(V, 0, "on_boundary")


N = MyExp(my_x, function_space=V).assemble_N()

print("N is resturned as a ", type(N), 'so I need to convert it to ufl')


N_new = Function(V)
N_new.dat.data[:] = N[:]


F = (inner(grad(u), grad(v)) - inner(N_new, v)) * dx
solve(F == 0, u, bcs=bcs)

N is resturned as a  <class 'numpy.ndarray'> so I need to convert it to ufl


# Vectors

## Use Numpy Absolute Value in Weak Form

In [71]:
class MyAbsVect(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""
      a = self.ufl_operands[0]
      N = np.abs(a)
      return N

In [72]:
V = VectorFunctionSpace(mesh, "CG", 1)

f = Function(V).interpolate(as_vector( [xi**2 - 1, eta**2 - 1] ))

N = MyAbsVect(f, function_space =  V).assemble_N()

print("N is")
print(N)
print()
print("N is resturned as a ", type(N), 'so I need to convert it to ufl')
print()


Nnew = as_vector([N[0], N[1]]) # is this the best appraoch?


N is
[Abs(Indexed(Coefficient(WithGeometry(FunctionSpace(<firedrake.mesh.MeshTopology object at 0x7d4e16b66c80>, VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), name=None), Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 244)), 499), MultiIndex((FixedIndex(0),))))
 Abs(Indexed(Coefficient(WithGeometry(FunctionSpace(<firedrake.mesh.MeshTopology object at 0x7d4e16b66c80>, VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), name=None), Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 244)), 499), MultiIndex((FixedIndex(1),))))]

N is resturned as a  <class 'numpy.ndarray'> so I need to convert it to ufl



In [73]:
u = Function(V)
v = TestFunction(V)
f = Function(V).interpolate(as_vector([xi**2, 1]))
g = Function(V).interpolate(as_vector([2, 1]))

bcs = DirichletBC(V, 0, "on_boundary")


F = (inner(grad(u), grad(v)) - inner(Nnew, v)) * dx

solve(F == 0, u, bcs=bcs)

## Use Numpy Multiplication in the Weak Form

In [74]:
# multiply

class MyMultVect(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""
      a, b = self.ufl_operands
      N = np.multiply(a, b)
      return N

In [75]:
# now lets solve a problem

xi, eta = SpatialCoordinate(mesh)
u = Function(V)
v = TestFunction(V)
f = Function(V).interpolate(as_vector([xi**2, 1]))
g = Function(V).interpolate(as_vector([2, 1]))

bcs = DirichletBC(V, 0, "on_boundary")


N = MyMultVect(f, g,  function_space =  V).assemble_N()

print("N is")
print(N)
print()
print("N is resturned as a ", type(N), 'so I need to convert it to ufl')
print()


Nnew = as_vector([N[0], N[1]]) # is this the best appraoch?

F = (inner(grad(u), grad(v)) - inner(Nnew, v)) * dx

solve(F == 0, u, bcs=bcs)

N is
[Product(Indexed(Coefficient(WithGeometry(FunctionSpace(<firedrake.mesh.MeshTopology object at 0x7d4e16b66c80>, VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), name=None), Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 244)), 520), MultiIndex((FixedIndex(0),))), Indexed(Coefficient(WithGeometry(FunctionSpace(<firedrake.mesh.MeshTopology object at 0x7d4e16b66c80>, VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), name=None), Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 244)), 524), MultiIndex((FixedIndex(0),))))
 Product(Indexed(Coefficient(WithGeometry(FunctionSpace(<firedrake.mesh.MeshTopology object at 0x7d4e16b66c80>, VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), name=None), Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 244)), 520), MultiIndex((FixedIndex(1),))), Indexed(Coefficient(WithGeometry(FunctionSpace(<firedrake.mesh.MeshTopology object at 0x7d4e16b66c80>, VectorElem

## Use Numpy sin(x) In the Weak Form


In [76]:
class MySinVect(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""
      my_x = self.ufl_operands[0]

      # N = np.sin(my_x)
      # above fails because of a
      # TypeError: loop of ufunc does not support argument 0 of type Function which has no callable sin method

      N = np.sin(my_x.dat.data) # apparently I need to convert to numpy array !?

      return N

In [77]:
# now lets solve a problem

u = Function(V)
v = TestFunction(V)
f = Function(V).interpolate(as_vector([xi**2, 1]))

bcs = DirichletBC(V, 0, "on_boundary")


N = MySinVect(f,  function_space =  V).assemble_N()

print("N is")
print(N)
print()
print("N is resturned as a ", type(N), 'so I need to convert it to ufl')
print()


# converting to ufl

Nnew = Function(V)
Nnew.dat.data[:] = N[:]


F = (inner(grad(u), grad(v)) - inner(Nnew, v)) * dx

solve(F == 0, u, bcs=bcs)

N is
[[0.         0.84147098]
 [0.         0.84147098]
 [0.00173611 0.84147098]
 ...
 [0.84147098 0.84147098]
 [0.79463298 0.84147098]
 [0.84147098 0.84147098]]

N is resturned as a  <class 'numpy.ndarray'> so I need to convert it to ufl



## Using Numpy exp(x) in the Weak Form

In [78]:
class MyExpVect(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""
      my_x = self.ufl_operands[0]

      # N = np.exp(my_x)
      # above fails because of a
      # TTypeError: loop of ufunc does not support argument 0 of type Indexed which has no callable exp method

      N = np.sin(my_x.dat.data) # apparently I need to convert to numpy array !?

      return N

In [79]:
# now lets solve a problem

u = Function(V)
v = TestFunction(V)
f = Function(V).interpolate(as_vector([xi**2, 1]))

bcs = DirichletBC(V, 0, "on_boundary")


N = MyExpVect(f,  function_space =  V).assemble_N()

print("N is")
print(N)
print()
print("N is resturned as a ", type(N), 'so I need to convert it to ufl')
print()


# converting to ufl

Nnew = Function(V)
Nnew.dat.data[:] = N[:]


F = (inner(grad(u), grad(v)) - inner(Nnew, v)) * dx

solve(F == 0, u, bcs=bcs)

N is
[[0.         0.84147098]
 [0.         0.84147098]
 [0.00173611 0.84147098]
 ...
 [0.84147098 0.84147098]
 [0.79463298 0.84147098]
 [0.84147098 0.84147098]]

N is resturned as a  <class 'numpy.ndarray'> so I need to convert it to ufl



# Matrices

## Use Numpy's Matrix Multiply in the Weak Form (WORKS ?)

In [80]:
class MyMultMatrix(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""
      a, b = self.ufl_operands
      N = np.multiply(a, b)
      return N

In [81]:
V = FunctionSpace(mesh, "CG", 2)
W = VectorFunctionSpace(mesh, "CG", 2)

f1 = Function(V).interpolate(xi**2 - 1)
f2 = Function(V).interpolate(eta**2 + 1)
f3 = Function(V).interpolate(xi**2 + 1 )
f4 = Function(V).interpolate(eta**2 - 1)
f = as_matrix( [[f1, f2], [f3, f4]] )

g1 = Function(V).interpolate(1)
g2 = Function(V).interpolate(2)
g3 = Function(V).interpolate(3)
g4 = Function(V).interpolate(4)
g = as_matrix( [[g1, g2], [g3, g4]] )

N = MyMultMatrix(f, g, function_space =  V).assemble_N()


print(N)
print()
print("N is of type", type(N), "without any conversions")

{ A | A_{i_{261}, i_{262}} = sum_{i_{263}} ([
  [w₅₆₁, w₅₆₅],
  [w₅₆₉, w₅₇₃]
])[i_{261}, i_{263}] * ([
  [w₅₇₇, w₅₈₁],
  [w₅₈₅, w₅₈₉]
])[i_{263}, i_{262}]  }

N is of type <class 'ufl.tensors.ComponentTensor'> without any conversions


In [82]:
u = Function(W)
v = TestFunction(W)
bcs = DirichletBC(W, 0, "on_boundary")

F = inner( grad(v), dot( N, grad(u)) ) * dx

solve(F == 0, u, bcs=bcs)

## Use Numpy's Component Wise Absolute Value in the Weak Form (FAILS)

In [83]:
class MyAbsMatrix(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""
      a = self.ufl_operands[0]
      N = np.abs(a)
      return N

In [84]:
V = FunctionSpace(mesh, "CG", 2)
W = VectorFunctionSpace(mesh, "CG", 2)
f1 = Function(V).interpolate(xi**2 - 1)
f2 = Function(V).interpolate(eta**2 + 1)
f3 = Function(V).interpolate(xi**2 + 1 )
f4 = Function(V).interpolate(eta**2 - 1)
f = as_matrix( [[f1, f2], [f3, f4]] )

N = MyAbsMatrix(f, function_space =  V).assemble_N()


print(N)
print()
print("N is of type", type(N), "without any conversions")

|([
  [w₆₁₄, w₆₁₈],
  [w₆₂₂, w₆₂₆]
])|

N is of type <class 'ufl.algebra.Abs'> without any conversions


In [85]:
u = Function(W)
v = TestFunction(W)
bcs = DirichletBC(V, 0, "on_boundary")

F = inner( grad(v), dot( N, grad(u)) ) * dx

# solve(F == 0, u, bcs=bcs) # FAILS
# ValueError: Expecting scalar arguments.

## Let's Use a Scipy Tool (FAILS)

### Similar To What I Have In My Hessian Code

In [86]:
class MyInvMatrix(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""

      a1, a2, a3, a4 = self.ufl_operands

      points = len(a1.dat.data)
      result = np.zeros((2, 2, points))

      for i in range(points):

        curr_a = np.array([[a1.dat.data[i], a2.dat.data[i]], [a3.dat.data[i], a4.dat.data[i]]])
        result[:, :, i] = sci.linalg.inv(curr_a)

      return result

In [87]:
V = FunctionSpace(mesh, "CG", 2)
W = VectorFunctionSpace(mesh, "CG", 2)


f1 = Function(V).interpolate(xi**2 - 1)
f2 = Function(V).interpolate(1)
f3 = Function(V).interpolate(xi**2 + 1 )
f4 = Function(V).interpolate(2)


f = as_matrix( [[f1, f2], [f3, f4]] )


N = MyInvMatrix(f1, f2, f3, f4, function_space =  V).assemble_N()



print("N is of type", type(N), "so we need to convert to ufl")
print()


N1 = Function(V)
N2 = Function(V)
N3 = Function(V)
N4 = Function(V)
N1.dat.data[:] = N[0, 0, :]
N2.dat.data[:] = N[0, 1, :]
N3.dat.data[:] = N[1, 0, :]
N4.dat.data[:] = N[1, 1, :]
Nnew = as_matrix([ [ N1, N2 ] , [ N3, N4 ]] )


print("N is now of type", type(Nnew))




N is of type <class 'numpy.ndarray'> so we need to convert to ufl

N is now of type <class 'ufl.tensors.ListTensor'>


In [88]:
u = Function(W)
v = TestFunction(W)
bcs = DirichletBC(W, 0, "on_boundary")

F = inner( grad(v), dot( Nnew, grad(u)) ) * dx

solve(F == 0, u, bcs=bcs)

### What I Would Like to Have (fails tho)

In [89]:
class MyInvMatrix(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""

      a = self.ufl_operands[0]

      N = sci.linalg.inv(a)

      # fails - causes ValueError: object arrays are not supported

      return N

In [90]:
V = FunctionSpace(mesh, "CG", 2)
W = VectorFunctionSpace(mesh, "CG", 2)


f1 = Function(V).interpolate(xi**2 - 1)
f2 = Function(V).interpolate(eta**2 + 1)
f3 = Function(V).interpolate(xi**2 + 1 )
f4 = Function(V).interpolate(eta**2 - 1)


f = as_matrix( [[f1, f2], [f3, f4]] )


N = MyInvMatrix(f, function_space =  V).assemble_N()


print(N)
print()
print("N is of type", type(N), "without any conversions")

ValueError: object arrays are not supported

In [ ]:
u = Function(W)
v = TestFunction(W)
bcs = DirichletBC(W, 0, "on_boundary")

F = inner( grad(v), dot( N, grad(u)) ) * dx

solve(F == 0, u, bcs=bcs)